# Modifies data from TOGA* as specified below

In [1]:
# The dataset is located at : RQ2/toga-model-inputs-outputs. 
# input.csv that's the input file where each row contains three information 
    # 1. focus method (the method being tested) 
    # 2, a test case with test assertions (you will need to remove that) 
    # 3) docstring (not necessary for SEER).
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re

folder_dir = './toga-model-inputs-outputs'

In [37]:
colnames = ['dataset', 'project', 'bug_id', 'C', 'T', 'docstring']
df = pd.DataFrame(columns=colnames)
folder_names = os.listdir(folder_dir)

for project in folder_names:
    project_path = os.path.join(folder_dir, project)
    temp_df = pd.read_csv(project_path + '/inputs.csv')
    temp_df.columns = ['C', 'T', 'docstring']
    # TODO some data is missing code cases such as toga-model-inputs-outputs/commons-imaging-1.0-alpha3-src/inputs.csv
    temp_df = temp_df.dropna()
    # TODO
    temp_df['dataset'] = 'toga*'
    temp_df['project'] = project
    temp_df['bug_id'] = "-1"
    #     "dataset": "Real Faults",
    #     "project": "Chart",
    #     "bug_id": "16",
    df = pd.concat([df, temp_df], ignore_index=True, axis=0)

df['label'] = 'P'
df = df.drop('docstring', axis=1)
df['T'] = df.apply(lambda row: re.sub(r'\s*assert.*', "", row['T']), axis=1)
for col in ['C', 'T']:
    df[col] = df[col].astype(str)
    df[col] = df.apply(lambda row: re.sub(r'\s\s*', " ", row[col].strip()), axis=1)
    df[col] = df.apply(lambda row: re.sub(r'\n', "", row[col]), axis=1)

df.to_json('./phase3.json', orient='index', indent=4)
df

,dataset,project,bug_id,C,T,label
0,toga*,commons-imaging-1.0-alpha3-src,-1,public boolean isProgressive() { return progre...,public void test21() throws Throwable { Linked...,P
1,toga*,commons-imaging-1.0-alpha3-src,-1,public int getPhysicalWidthDpi() { return phys...,public void test22() throws Throwable { Linked...,P
2,toga*,commons-imaging-1.0-alpha3-src,-1,public float getPhysicalHeightInch() { return ...,public void test23() throws Throwable { Linked...,P
3,toga*,commons-imaging-1.0-alpha3-src,-1,public int getNumberOfImages() { return number...,public void test24() throws Throwable { Linked...,P
4,toga*,commons-imaging-1.0-alpha3-src,-1,public String getMimeType() { return mimeType; },public void test25() throws Throwable { Linked...,P
...,...,...,...,...,...,...
169094,toga*,commons-dbutils-1.7,-1,public float getNullFloat() { return this.null...,public void test324() throws Throwable { Resul...,P
169095,toga*,commons-dbutils-1.7,-1,public double getNullDouble() { return this.nu...,public void test325() throws Throwable { Resul...,P
169096,toga*,commons-dbutils-1.7,-1,public short getNullShort() { return this.null...,public void test326() throws Throwable { Resul...,P
169097,toga*,commons-dbutils-1.7,-1,public int getNullInt() { return this.nullInt; },public void test327() throws Throwable { Resul...,P


In [3]:
max(df.apply(lambda row: len(row['C']), axis=1))

13532

In [4]:
max(df.apply(lambda row: len(row['T']), axis=1))

6637

# ddd

In [8]:
# import h5py
# filename = "code_test.h5"

# with h5py.File(filename, "r") as f:
#     # Print all root level object names (aka keys) 
#     # these can be group or dataset names 
#     print("Keys: %s" % f.keys())
#     # get first object name/key; may or may NOT be a group
#     a_group_key = list(f.keys())[0]

#     # get the object type for a_group_key: usually group or dataset
#     print(type(f[a_group_key])) 

#     # If a_group_key is a group name, 
#     # this gets the object names in the group and returns as a list
#     data = list(f[a_group_key])

#     # If a_group_key is a dataset name, 
#     # this gets the dataset values and returns as a list
#     data = list(f[a_group_key])
#     # preferred methods to get dataset values:
#     ds_obj = f[a_group_key]      # returns as a h5py dataset object
#     ds_arr = f[a_group_key][()]  # returns as a numpy array

Keys: <KeysViewHDF5 ['indices', 'phrases']>
<class 'h5py._hl.dataset.Dataset'>


# Removing try catch blocks and making failing tests

In [11]:
string = df.loc[1308]['T']
string

'public void test12() throws Throwable { PnmImageParser pnmImageParser0 = new PnmImageParser(); SimpleBufferedImageFactory simpleBufferedImageFactory0 = new SimpleBufferedImageFactory(); BufferedImage bufferedImage0 = simpleBufferedImageFactory0.getColorBufferedImage(100, 100, true); PnmImagingParameters pnmImagingParameters0 = pnmImageParser0.getDefaultParameters(); // Undeclared exception! try { pnmImageParser0.writeImage(bufferedImage0, (OutputStream) null, pnmImagingParameters0); fail("Expecting exception: NullPointerException"); } catch(NullPointerException e) { // // no message in exception (getMessage() returned null) // verifyException("org.apache.commons.imaging.formats.pnm.PamWriter", e); } }'

In [45]:
df_try.loc[1309]['T']

'public void test13() throws Throwable { PngImageParser pngImageParser0 = new PngImageParser(); byte[] byteArray0 = new byte[0]; PngImagingParameters pngImagingParameters0 = new PngImagingParameters();  pngImageParser0.getMetadata(byteArray0, pngImagingParameters0);   }'

In [39]:
r_try = r'try\s*{'
r_fail = r'fail\([^;]*;\s*}'
r_except = r'catch\([^}]*\s*}'

try_except_regex = [r_try, r_fail, r_except]
df_try = df

In [13]:
# for regex in try_except_regex:
#     string = re.sub(regex, "", string)
# string

In [44]:
for regex in try_except_regex:
    df_try['label'] = df_try.apply(lambda row: "F" if (re.search(regex, row['T']) != None) else row['label'], axis=1)
    df_try['T'] = df_try.apply(lambda row: re.sub(regex, "", row['T']), axis=1)

In [46]:
df_try.to_json('./phase3_no_try_except.json', orient='index', indent=4)
# df_try['T'].to_csv('./test.csv')